In [1]:
%reload_ext setup

INFO:setup:IPython environment configured successfully!


## Introduction to Agents

In [2]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt="You are a helpful assistant",
    critic_prompt="You are a critic, giving useful fedback. If happy with the result, say 'APPROVE'. If not, give feedback."
)
# display(agents)

# Cell 3: Run the chat with simplified output
message = "Hello, tell a joke?"
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):  # Only show actual text messages between agents
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):  # Show final result
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

user: Hello, tell a joke?
matcher: Why don't skeletons fight each other? They don't have the guts!
critic: APPROVE
Task completed. Stop reason: Text 'APPROVE' mentioned


## Demo Example (1 registrants)

In [10]:
# Cell 3: Read and combine prompt files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

general_context = read_file('../prompts/general_context.txt')
matching_rules = read_file('../prompts/matching_rules.txt')
distribution_rules = read_file('../prompts/distribution_rules.txt')
matcher_prompt = read_file('../prompts/matcher/matcher_1.txt')
critic_prompt = read_file('../prompts/critic/critic_1.txt')

combined_matcher_prompt = matcher_prompt.format(general_context=general_context, matching_rules=matching_rules, distribution_rules=distribution_rules)
combined_critic_prompt = critic_prompt.format(general_context=general_context, matching_rules=matching_rules, distribution_rules=distribution_rules)

In [12]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agents
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agents(
    state_path="model_config.yaml",
    matcher_prompt=combined_matcher_prompt,
    critic_prompt=combined_critic_prompt
)

# Cell 3: Run the chat with matching task
message = (
    "Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. "
    "Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), "
    "and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. "
    "Output matches in JSON format as per the matching rules."
)
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

user: Load data from '../data/demo/registrations.json' and '../data/demo/offers.json'. Match all registrations to suppliers based on ZIP code, product needs (e.g., panels), and supplier capacity. Ensure proportional distribution across suppliers with overlapping service areas. Output matches in JSON format as per the matching rules.
critic: The provided matching data needs verification according to the defined rules. Let's evaluate the matching step-by-step.

### Step-by-Step Verification

1. **Matching by ZIP Code**:
   - The registration ZIP code is `55409`.
   - Based on the offers, the suppliers serving `55409` are:
     - All Energy Solar (servicing ZIPs range including `55409`)
     - iSolar (servicing ZIPs range including `55409`)

2. **Supplier Capacity**:
   - All Energy Solar: Capacity of 100.
   - iSolar: Capacity of 100.
   - TruNorthSolar is not applicable for `55409` as it does not service this area.

3. **Product Needs (Panels)**:
   - The registered campaign requires pa

INFO:openai._base_client:Retrying request to /chat/completions in 1.020000 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 24.080000 seconds


matcher: Here's the JSON-formatted match for registration based on details from suppliers and using specified matching and distribution rules:

```json
[{
  "registration_id": "SPUS62763",
  "campaign_id": "MN1",
  "supplier_id": "iSoalr",
  "panel_name": "Silfab HG",
  "panel_capacity": "410",
  "battery_name": "Tesla Powerwall 2",
  "battery_capacity": "13.5"
}]
```

This result reflects the best alignment according to available data, maintaining equal opportunity given ZIP and product needs. Adjustments for actual assignment metrics or dynamic real-time data would depend on additional contextual and momentary conditions.


INFO:openai._base_client:Retrying request to /chat/completions in 5.574000 seconds


critic: APPROVE

```json
[{
  "registration_id": "SPUS62763",
  "campaign_id": "MN1",
  "supplier_id": "iSoalr",
  "panel_name": "Silfab HG",
  "panel_capacity": "410",
  "battery_name": "Tesla Powerwall 2",
  "battery_capacity": "13.5"
}]
```
Task completed. Stop reason: Text 'APPROVE' mentioned
